# Predicting house prices using k-nearest neighbors regression

In this notebook, you will implement k-nearest neighbors regression. You will:

Find the k-nearest neighbors of a given query input Predict the output for the query input using the k-nearest neighbors Choose the best value of k using a validation set

In [88]:
import numpy as np
import pandas as pd
import os
from IPython.display import display

# dictionary with dataset column names and their corresponding data types
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 
              'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 
              'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 
              'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

Load the train, test and validate datasets in pandas dataframe

In [89]:
# change directory to the location where the data for the notebook is located
os.chdir('D:\Anupam_Technical\Code\ML\DS_ML_Projects\Regression\data\knn')
train = pd.read_csv('kc_house_data_small_train.csv', dtype = dtype_dict)
print('Top 5 rows of training dataset')
display(train.head())
test = pd.read_csv('kc_house_data_small_test.csv', dtype = dtype_dict)
validate = pd.read_csv('kc_house_data_validation.csv', dtype = dtype_dict)

Top 5 rows of training dataset


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


This function extracts input and output features from a dataframe and return a tuple of input features numpy 2d array and output feature vector (numpy 1d array). Note that we need to add a column vector of all ones as the first column of the input feature matrix

In [90]:
# add a column vector of all ones to the begining of a feature matrix
def add_one_vector(X):
    one_vector = np.ones(len(X)).reshape(len(X), 1)
    return np.concatenate((one_vector, X), axis=1)


def extract_features(df, datatype_dict):
    """
    Extract input and output features from a dataframe and return a tuple of input features matrix and output 
    feature vector
    :param df: dataframe
    :param datatype_dict: dictionary with dataset column names and their corresponding data types
    :return: a tuple of input features matrix and output feature vector
    """    
    # remove the columns that are not numeric i.e. int, floats etc.
    # train.dtypes gives a pandas with index as column names and value as column data types. We filter this
    # series to remove columns of type object
    numeric_cols = pd.Series(train.dtypes).where(lambda col_dtype: col_dtype != 'object').dropna()
    feature_names = list(numeric_cols.keys().values)
    # price is the output variable
    feature_names.remove('price')
    # extract the input features from the dataframe as a numpy 2d array
    input_features = add_one_vector(df[feature_names].values)
    output_variable = df.loc[:, 'price'].values
    return input_features, output_variable

In [91]:
train_input_features, train_output = extract_features(train, dtype_dict)
cv_input_features, cv_output = extract_features(validate, dtype_dict)
test_input_features, test_output = extract_features(test, dtype_dict)

In computing distances, it is crucial to normalize features. Otherwise, for example, the ‘sqft_living’ feature (typically on the order of thousands) would exert a much larger influence on distance than the ‘bedrooms’ feature (typically on the order of ones). We divide each column of the training feature matrix by its 2-norm, so that the transformed column has unit norm.

IMPORTANT: Make sure to store the norms of the features in the training set. The features in the test and validation sets must be divided by these same norms, so that the training, test, and validation sets are normalized consistently.

In [92]:
def normalize_features(input_features):
    norm = np.sqrt(np.sum(input_features**2, axis=0))
    normalized_features = input_features / norm
    return normalized_features, norm

norm_train_input_features, train_norm = normalize_features(train_input_features)
norm_cv_input_features = cv_input_features / train_norm
norm_test_input_features = test_input_features / train_norm

# Compute a single distance

To start, let's just explore computing the “distance” between two given houses. We will take our query house to be the first house of the test set and look at the distance between this house and the 10th house of the training set.

To see the features associated with the query house, print the first row (index 0) of the test feature matrix. You should get an 18-dimensional vector whose components are between 0 and 1. Similarly, print the 10th row (index 9) of the training feature matrix.

In [93]:
print(norm_test_input_features[0])
print(norm_train_input_features[9])

[ 0.01345102  0.01551285  0.01807473  0.01759212  0.00160518  0.017059
  0.          0.05102365  0.0116321   0.01564352  0.01362084  0.02481682
  0.01350306  0.          0.01345387 -0.01346922  0.01375926  0.0016225 ]
[ 0.01345102  0.01163464  0.00602491  0.0083488   0.00050756  0.01279425
  0.          0.          0.01938684  0.01390535  0.0096309   0.
  0.01302544  0.          0.01346821 -0.01346251  0.01195898  0.00156612]


What is the Euclidean distance between the query house and the 10th house of the training set?

In [94]:
np.sqrt(np.sum((norm_train_input_features[9] - norm_test_input_features[0])**2))

0.05972359371398078

Of course, to do nearest neighbor regression, we need to compute the distance between our query house and all houses in the training set.

To visualize this nearest-neighbor search, let's first compute the distance from our query house (features_test[0]) to the first 10 houses of the training set (norm_train_input_features[0:10]) and then search for the nearest neighbor within this small set of houses. Through restricting ourselves to a small set of houses to begin with, we can visually scan the list of 10 distances to verify that our code for finding the nearest neighbor is working.

Write a loop to compute the Euclidean distance from the query house to each of the first 10 houses in the training set.

Quiz Question: Among the first 10 training houses, which house is the closest to the query house?

In [95]:
distance = {}
for i in range(10):
    distance[i] = np.sqrt(np.sum((norm_train_input_features[i] - norm_test_input_features[0])**2))
print(distance)

{0: 0.06027470916295592, 1: 0.08546881147643746, 2: 0.06149946435279315, 3: 0.05340273979294363, 4: 0.05844484060170442, 5: 0.059879215098128345, 6: 0.05463140496775461, 7: 0.055431083236146074, 8: 0.052383627840220305, 9: 0.05972359371398078}


In [96]:
distance_sorted = sorted(distance.items(), key=lambda item: item[1])
print(distance_sorted)

[(8, 0.052383627840220305), (3, 0.05340273979294363), (6, 0.05463140496775461), (7, 0.055431083236146074), (4, 0.05844484060170442), (9, 0.05972359371398078), (5, 0.059879215098128345), (0, 0.06027470916295592), (2, 0.06149946435279315), (1, 0.08546881147643746)]


# Perform one nearest neighbour regression

Now that we have the element-wise differences, it is not too hard to compute the Euclidean distances between our query house and all of the training houses. First, write a single-line expression to define a variable ‘diff’ such that ‘diff[i]’ gives the element-wise difference between the features of the query house and the i-th training house.

To test your code, print diff[-1].sum(), which should be -0.0934339605842.

In [97]:
diff = norm_train_input_features[:] - norm_test_input_features[0]
diff[-1].sum()

-0.09343399874654643

The next step in computing the Euclidean distances is to take these feature-by-feature differences in ‘diff’, square each, and take the sum over feature indices. That is, compute the sum of squared feature differences for each training house (row in ‘diff’).

By default, ‘np.sum’ sums up everything in the matrix and returns a single number. To instead sum only over a row or column, we need to specifiy the ‘axis’ parameter described in the np.sum documentation. In particular, ‘axis=1’ computes the sum across each row.

In [98]:
def compute_distance(training_examples, query_house):
    """
    Vectorized implementation of calculating the distance of a query house from each of the training examples
    :param training_examples: a matrix or numpy 2d array consisting of training data (input features)
    :param query_house: the query house
    :return: numpy 2d array whose first column is the training row index and second column is the distance from
    the query house
    """
    # subtract the query house row from each training example row
    diff_matrix = training_examples - query_house
    # now for each row in the matrix (which corresponds to each training example), calculate the sum of
    # squares of feature values ( this is done by using axis = 1 in the 2d array )
    distance = np.sqrt(np.sum(diff_matrix**2, axis=1))
    index = np.arange(0, len(distance))    
    return np.concatenate((index.reshape(-1, 1), distance.reshape(-1, 1)), axis=1)

Compute the distance of the third test example from each of the training examples. Then find out which training example is the closest to the query house. Note that the expected value is row index 382 has the min. distance of 0.0028604955575117085

In [99]:
rowindex_distance = compute_distance(norm_train_input_features[:], norm_test_input_features[2])

def get_min_distance_row_index(rowindex_distance):
    rowindex = rowindex_distance[:, 0]
    distance = rowindex_distance[:, 1]
    min_distance = np.amin(distance)
    min_distance_index = np.where(distance == min_distance)
    return rowindex[min_distance_index], min_distance

min_row_index, min_distance = get_min_distance_row_index(rowindex_distance)
print(int(min_row_index[0]), min_distance)

382 0.0028604955575117085


What is the predicted value of the query house (third test example) based on 1-nearest neighbor regression?

In [100]:
# Since the query house ( i.e. the third test example) is closest to the 382nd training example, hence the predicted 
# value of the query house will be same as the price of the 382nd training example
train_output[382]

249000.0

# Perform k-nearest neighbor regression

Using the functions above, implement a function that takes in

the value of k; the feature matrix for the instances; and the feature of the query and returns the indices of the k closest training houses. For instance, with 2-nearest neighbor, a return value of [5, 10] would indicate that the 6th and 11th training houses are closest to the query house.

In [101]:
def k_nearest_neighbours(k, training_examples, query_house):
    rownumber_distance = compute_distance(training_examples, query_house)
    # sort the 2d array on the index column in ascending order
    # You can call .argsort() on the column you want to sort, and it will give you an array of row indices 
    # that sort that particular column which you can pass as an index to your original array.
    rownumber_distance_sorted = rownumber_distance[rownumber_distance[:, 1].argsort()]
    return rownumber_distance_sorted[0:k, :]

Take the query house to be third house of the test set (norm_test_input_features[2]). What are the indices of the 4 training houses closest to the query house?

In [102]:
knn_rowindex_distance = k_nearest_neighbours(4, norm_train_input_features[:], norm_test_input_features[2])
print(knn_rowindex_distance[:, 0].astype(int))

[ 382 1149 4087 3142]


Now that we know how to find the k-nearest neighbors, write a function that predicts the value of a given query house. For simplicity, take the average of the prices of the k nearest neighbors in the training set. The function should have the following parameters:

<ul>
    <li>the value of k;</li>
    <li>the feature matrix for the instances;</li>
    <li>the output values (prices) of the instances; and</li>
    <li>the feature of the query, whose price we’re predicting.</li>
</ul>
The function should return a predicted value of the query house.

In [103]:
def predict_house_price_custom(k, train_input_features, train_output, query_house):
    k_rowindex_distance = k_nearest_neighbours(k, train_input_features, query_house)
    # get the rowindex of the k nearest neighbours and get their corresponding prices
    k_rowindex = k_rowindex_distance[:, 0].astype(int)
    # get the mean of the k nearest house prices, this is the predicted price of the query house
    return np.mean(train_output[k_rowindex])

Make predictions for the first 10 houses in the test set, using k=10. What is the index of the house in this query set that has the lowest predicted value? What is the predicted value of this house?

In [104]:
query_house_predicted_price = []

for query_house_index in range(10):
    query_house = norm_test_input_features[query_house_index]
    predicted_price = predict_house_price_custom(10, norm_train_input_features, train_output, query_house)
    query_house_predicted_price.append((query_house_index, predicted_price))

# sort on the basis of predicted prices in ascending order    
sorted_query_house_predicted_price = sorted(query_house_predicted_price, key=lambda item:item[1])
print(sorted_query_house_predicted_price)
print('\nHouse index in test set of 10 houses with lowest predicted price: {}'
      .format(sorted_query_house_predicted_price[0][0]))

[(6, 350032.0), (3, 430200.0), (1, 431860.0), (9, 457235.0), (2, 460595.0), (8, 484000.0), (7, 512800.7), (5, 667420.0), (4, 766750.0), (0, 881300.0)]

House index in test set of 10 houses with lowest predicted price: 6


# Choosing the best value of k using a validation set

There remains a question of choosing the value of k to use in making predictions. Here, we use a validation set to choose this value. Write a loop that does the following:

For k in [1, 2, … 15]:

Make predictions for the VALIDATION data using the k-nearest neighbors from the TRAINING data. Compute the RSS on VALIDATION data Report which k produced the lowest RSS on validation data.

In [107]:
def get_optimized_kvalue(list_kvalues, predict_house_price):
    k_rss = []
    for k in list_kvalues:    
        queryhouseindex_predictedprice_actualprice = []
        for query_house_index in range(len(norm_cv_input_features)):
            query_house = norm_cv_input_features[query_house_index]
            actual_price = cv_output[query_house_index]
            predicted_price = predict_house_price(k, norm_train_input_features, train_output, query_house)
            queryhouseindex_predictedprice_actualprice.append([query_house_index, predicted_price, actual_price])
        queryhouseindex_predictedprice_actualprice = np.array(queryhouseindex_predictedprice_actualprice)            
        # now calculate the residual sum of squares ( (predicted value - actual value)**2 ) over the entire validation set
        price_diff = (queryhouseindex_predictedprice_actualprice[:, 1] - queryhouseindex_predictedprice_actualprice[:, 2])**2
        rss = np.sum(price_diff)
        k_rss.append((k, rss))   
        print('k: {} --> rss: {}'.format(k, rss))
    sorted_k_rss = sorted(k_rss, key=lambda item:item[1])    
    return sorted_k_rss[0][0]

optimized_k = get_optimized_kvalue(np.arange(15)+1, predict_house_price_custom)
print('\n The value of k that minimizes RSS on validation data is: {}'.format(optimized_k))


k: 1 --> rss: 105453830251561.0
k: 2 --> rss: 83445073504025.5
k: 3 --> rss: 72692096019202.56
k: 4 --> rss: 71946721652091.69
k: 5 --> rss: 69846517419718.6
k: 6 --> rss: 68899544353180.836
k: 7 --> rss: 68341973450051.09
k: 8 --> rss: 67361678735491.5
k: 9 --> rss: 68372727958976.09
k: 10 --> rss: 69335048668556.74
k: 11 --> rss: 69523855215598.83
k: 12 --> rss: 69049969587246.17
k: 13 --> rss: 70011254508263.69
k: 14 --> rss: 70908698869034.34
k: 15 --> rss: 71106928385945.16

 The value of k that minimizes RSS on validation data is: 8


# K Nearest Neighbours with scikit-learn

In [108]:
from sklearn.neighbors import KNeighborsRegressor

def predict_house_price_scikit(k, train_input_features, train_output, query_house):
    k10_regressor = KNeighborsRegressor(n_neighbors = k, weights='distance')
    k10_regressor.fit(train_input_features, train_output)    
    return k10_regressor.predict(query_house.reshape(1, -1))

predict_house_price_scikit(4, norm_train_input_features, train_output, norm_test_input_features[2])
optimized_k_scikit = get_optimized_kvalue(np.arange(15)+1, predict_house_price_scikit)
print('\n The value of k (using scikit nearest neighbors) that minimizes RSS on validation data is: {}'
      .format(optimized_k_scikit))


k: 1 --> rss: [1.05451198e+14]
k: 2 --> rss: [8.3482811e+13]
k: 3 --> rss: [7.26620105e+13]
k: 4 --> rss: [7.16405256e+13]
k: 5 --> rss: [6.95642264e+13]
k: 6 --> rss: [6.8640568e+13]
k: 7 --> rss: [6.7961639e+13]
k: 8 --> rss: [6.69584338e+13]
k: 9 --> rss: [6.76112457e+13]
k: 10 --> rss: [6.79035784e+13]
k: 11 --> rss: [6.79588549e+13]
k: 12 --> rss: [6.77070525e+13]
k: 13 --> rss: [6.84091455e+13]
k: 14 --> rss: [6.90942193e+13]
k: 15 --> rss: [6.9224199e+13]

 The value of k (using scikit nearest neighbors) that minimizes RSS on validation data is: 8
